In [ ]:
import numpy as np
import pandas as pd

# Task 1: Build two-period recombining binomial tree for stock price
# Real-world tie-in: Binomial tree approximates dynamic stock paths (e.g., TSLA over two periods like quarters). Recombining saves computation — real for option pricing trees in volatile stocks.
S0 = 100      # Initial stock price
u = 1.2       # Up factor (e.g., strong growth)
d = 1/u       # Down factor (risk-neutral symmetric for simplicity)
r = 0.05      # Risk-free rate per period
periods = 2

# Stock prices at each node (recombining: uu = ud * u, etc.)
# Node labeling: [t=0], [t=1: up, down], [t=2: uu, ud, dd]
stock_tree = np.zeros((periods + 1, periods + 1))
stock_tree[0, 0] = S0
for t in range(1, periods + 1):
    for i in range(t + 1):
        stock_tree[i, t] = S0 * (u ** (t - i)) * (d ** i)

print("Two-Period Binomial Stock Tree:")
print(pd.DataFrame(stock_tree.T, columns=['uu', 'ud', 'dd'][:periods+1], index=[f"t={t}" for t in range(periods+1)]))

# Task 2: Compute state prices ψ recursively backward
# Real-world: Backward induction prices options dynamically (core to lattice models). ψ_s are Arrow-Debreu prices per node—verify no-arb by repricing stock/bond.
# Risk-neutral probability q = (1 + r - d) / (u - d)
q = (1 + r - d) / (u - d)
print(f"\nRisk-neutral q: {q:.4f}")

# State prices ψ (cumulative from root, backward)
psi_tree = np.zeros((periods + 1, periods + 1))
psi_tree[0, 0] = 1.0  # Root

for t in range(1, periods + 1):
    for i in range(t + 1):
        # From parent nodes
        if i < t:
            psi_up = psi_tree[i, t-1] * q / (1 + r)
        else:
            psi_up = 0
        if i > 0:
            psi_down = psi_tree[i-1, t-1] * (1 - q) / (1 + r)
        else:
            psi_down = 0
        psi_tree[i, t] = psi_up + psi_down

print("\nState Prices ψ Tree:")
print(pd.DataFrame(psi_tree.T))

# Verify pricing stock and bond
bond_price = np.sum(psi_tree[:, -1])  # Bond pays 1 everywhere at T=2
stock_price = np.sum(psi_tree[:, -1] * stock_tree[:, -1])

print(f"\nVerification (t=0 prices):")
print(f"  Bond: {bond_price:.4f} (should be 1/(1+r)^2 ≈ {(1/(1+r)**2):.4f})")
print(f"  Stock: {stock_price:.4f} (should be {S0:.4f})")

# Task 3: Dynamically replicate European call and initial cost
# Real-world: Dynamic delta-hedging over periods—market makers rebalance TSLA options portfolios step-by-step to neutralize risk.
K = 100  # Strike

# Call payoffs at maturity (t=2)
call_terminal = np.maximum(stock_tree[:, -1] - K, 0)

# Backward induction for option values and deltas
option_tree = np.zeros_like(stock_tree)
delta_tree = np.zeros_like(stock_tree)
option_tree[:, -1] = call_terminal

for t in range(periods - 1, -1, -1):
    for i in range(t + 1):
        # Option value from children
        V_up = option_tree[i, t+1]
        V_down = option_tree[i+1, t+1] if i < t else option_tree[i, t+1]  # Recombining
        option_tree[i, t] = (q * V_up + (1 - q) * V_down) / (1 + r)
        
        # Delta: shares to hold = (V_up - V_down) / (S_up - S_down)
        S_up = stock_tree[i, t] * u
        S_down = stock_tree[i, t] * d
        delta_tree[i, t] = (V_up - V_down) / (S_up - S_down)

# Initial hedging cost = option value at root
initial_cost = option_tree[0, 0]

print(f"\nEuropean Call Tree Values:")
print(pd.DataFrame(option_tree.T))
print(f"\nDelta Tree (shares to hold):")
print(pd.DataFrame(delta_tree.T))

print(f"\nDynamic Replication Initial Cost (fair price): {initial_cost:.4f}")
print("Interpretation: Replicate by holding delta shares + borrowing at each node—cost matches risk-neutral price. In real multi-period hedging (e.g., weekly rebalance), approximates continuous delta for TSLA options.")

Two-Period Binomial Stock Tree:
        uu          ud         dd
t=0  100.0    0.000000   0.000000
t=1  120.0   83.333333   0.000000
t=2  144.0  100.000000  69.444444

Risk-neutral q: 0.5909

State Prices ψ Tree:
          0         1         2
0  1.000000  0.000000  0.000000
1  0.562771  0.389610  0.000000
2  0.316711  0.438523  0.151796

Verification (t=0 prices):
  Bond: 0.9070 (should be 1/(1+r)^2 ≈ 0.9070)
  Stock: 100.0000 (should be 100.0000)

European Call Tree Values:
           0    1    2
0  23.582766  0.0  0.0
1  24.761905  0.0  0.0
2  44.000000  0.0  0.0

Delta Tree (shares to hold):
     0    1    2
0  0.0  0.0  0.0
1  1.0  0.0  0.0
2  0.0  0.0  0.0

Dynamic Replication Initial Cost (fair price): 23.5828
Interpretation: Replicate by holding delta shares + borrowing at each node—cost matches risk-neutral price. In real multi-period hedging (e.g., weekly rebalance), approximates continuous delta for TSLA options.
